In [259]:
import numpy as np
import pandas as pd 

In [265]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://34.79.35.240:7687",
                              auth=("neo4j", "azerty5"))

In [266]:
from os import listdir
from os.path import isfile, join
mypath = "ethereum/data/transactions/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
A = pd.read_csv(mypath+onlyfiles[0], delimiter = ";")

In [291]:
class Transaction:

    """ Attribut de classe """

    def __init__(self, ligne_tableau):
        
        self.hash = ligne_tableau.hash
        self.from_addr = ligne_tableau.from_address
        self.to_addr = ligne_tableau.to_address
        self.value = float(ligne_tableau.value)*10**(-18)
        self.gas_info = (ligne_tableau.gas, ligne_tableau.gas_price)

        self.receip_status = "Succès" if ligne_tableau.receipt_status==1 else "Échec"
        self.blocknumber = ligne_tableau.block_number
        self.blockhash = ligne_tableau.block_hash
        self.date = ligne_tableau.block_timestamp
        self.contract_address = ligne_tableau.receipt_contract_address
        self.transaction_type = ""
    
    """ Méthodes """
    
    def is_smartcontract(self):
        
        if self.value == '0' : 
            return "Éxecution de contrat"
        elif not pd.isnull(self.contract_address) : 
            return  "Création de contrat"
        else : 
            return 'Transaction'

        


    def add_transac(self,tx, name, friend_name, value, time, transactionhash, transactionstate, gas, gasprice, issc, blockhash):
        if name==friend_name:
            tx.run("MERGE (a:Address {Hash: $name}) "
            "MERGE (a)-[:SEND {Valeur : $value , Time_stamp : $time,Hash:$transactionhash, État : $transactionstate, Gas: $gas, Gas_price : $gasprice, Est_un_smart_contract : $issc, Block_hash : $blockhash}]->(a)",
            name=name, value=value, time=time,transactionhash=transactionhash,transactionstate=transactionstate, gas=gas, gasprice=gasprice, issc=issc, blockhash=blockhash)
        else:
            tx.run("MERGE (a:Address {Hash: $name}) "
                   "MERGE (b:Address {Hash: $friend_name})"
            "MERGE (a)-[:SEND {Valeur : $value,Time_stamp:$time, Hash:$transactionhash, État:$transactionstate, Gas: $gas, Gas_price : $gasprice, Est_un_smart_contract : $issc, Block_hash : $blockhash}]->(b)",
            name=name, value=value, friend_name=friend_name, time=time,transactionhash=transactionhash,transactionstate=transactionstate, gas=gas, gasprice=gasprice, issc=issc, blockhash=blockhash)
         #transactionstate : $transactionstate, gas : $gas, gasprice : $gasprice, issc : $issc, blockhash : $blockhash}]->(b)",

    def existe(self,tx,hash):
        query = ("MATCH (a:Address) WHERE a.Hash = $hash "
                    "RETURN a.Hash")
        for record in tx.run(query, hash=hash):
            return record['a.Hash'] == hash 
        return False

    def create(self,tx,hash):
        def nickname(hash):
            return hash[0:5]
        query = ("CREATE ("+'A'+hash+":Address {Hash : $hash} )")
        tx.run(query , hash=hash)

    def toCypher(self):
        
#         if not driver.session().execute_read(self.existe, self.from_addr):
#             driver.session().execute_write(self.create, self.from_addr)

#         if not driver.session().execute_read(self.existe, self.to_addr):
#             driver.session().execute_write(self.create, self.to_addr)
        
        driver.session().execute_write(self.add_transac,self.from_addr, self.to_addr, self.value,self.date, self.hash, str(self.receip_status), str(self.gas_info[0]), str(self.gas_info[1]), str(self.is_smartcontract()), str(self.blockhash))

In [ ]:
import time
def f(x) :
    Transaction(x).toCypher()

for i in range(3,len(onlyfiles)):
    t1=time.clock()
    file = onlyfiles[i]
    A = pd.read_csv(mypath+onlyfiles[i], delimiter = ";")
    A.apply(f, axis = 1)
    t2 = time.clock()
    print(t2-t1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
